
# Exercise 3: Fine-Tuning Pretrained Transformer on Text Classification Task

In this lab, you will apply the concepts learned by fine-tuning a pre-trained Transformer model on a text classification task using the Hugging Face `transformers` library.

## Objectives:
- Learn to load a pre-trained model from Hugging Face.
- Fine-tune the model on a text classification dataset.
- Evaluate and save the fine-tuned model.

### Instructions:
Follow the steps below to complete the lab.


## Installing Necessary Libraries

In this lab, we will use the following Python libraries:

- **Transformers**: Hugging Face's `transformers` library provides pre-trained models for various Natural Language Processing (NLP) tasks. We will use this to load and fine-tune a pre-trained transformer model.
  
- **Datasets**: Hugging Face's `datasets` library allows easy access to a wide range of datasets and provides tools for efficient data processing.
  
- **Scikit-learn**: A widely-used library for machine learning, which includes tools for metrics, evaluation, and preprocessing. In this lab, we'll use it to calculate evaluation metrics such as accuracy, precision, recall, and F1 score.
  
- **Accelerate**: This library allows seamless usage of different hardware (CPU, GPU) for training, helping to speed up the training process by utilizing all available resources efficiently.

You can install these libraries using the following command:

```bash
!pip install transformers datasets scikit-learn accelerate


In [ ]:

# Install necessary libraries
!pip install transformers datasets scikit-learn accelerate


## Step 1: Loading and Preparing the Dataset

In [ ]:

# Importing necessary libraries
from datasets import load_dataset

# Load IMDb dataset
dataset = load_dataset('imdb')
print(dataset)


## Step 2: Tokenizing the Dataset

In [ ]:

# Importing tokenizer from Hugging Face
from transformers import AutoTokenizer

# Load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Define tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(tokenized_datasets)


## Step 3: Setting up the Trainer

In [ ]:

# Import pre-trained model and Trainer utilities
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained BERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    logging_dir='./logs',
)

# Create trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'].shuffle().select(range(1000)),
    eval_dataset=tokenized_datasets['test'].shuffle().select(range(1000)),
)


## Step 4: Training the Model

In [ ]:

# Start training
trainer.train()


## Step 5: Evaluating the Model

### Evaluation Metrics

we will evaluate the performance of the fine-tuned model using the following metrics:

- **Accuracy**: This measures the percentage of correctly predicted labels out of all predictions. It is useful for understanding the overall correctness of the model. 
  - **Formula**: 
    $$
    \text{Accuracy} = \frac{\text{Correct Predictions}}{\text{Total Predictions}}
    $$

- **Precision**: Precision measures how many of the positive predictions made by the model are actually correct. It is especially important when the cost of false positives is high (i.e., incorrectly labeling negative examples as positive).
  - **Formula**: 
    $$
    \text{Precision} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}
    $$

- **Recall**: Recall (also known as sensitivity or true positive rate) measures how many actual positive cases the model is able to correctly identify. It is important when the cost of missing positive examples (false negatives) is high.
  - **Formula**: 
    $$
    \text{Recall} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}
    $$

- **F1 Score**: The F1 score is the harmonic mean of precision and recall, providing a balance between the two. It is particularly useful when you need to balance both false positives and false negatives.
  - **Formula**: 
    $$
    \text{F1} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
    $$


In [ ]:

# Importing evaluation metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define function to compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

# Update trainer to include custom metrics
trainer.compute_metrics = compute_metrics

# Evaluate the model
eval_result = trainer.evaluate()
print(eval_result)


## Step 6: Saving the Fine-Tuned Model

In [ ]:

# Save the fine-tuned model and tokenizer
trainer.save_model('my-fine-tuned-bert')
tokenizer.save_pretrained('my-fine-tuned-bert')


## Step 7: Loading and Testing the Saved Model

In [ ]:

# Load saved model and tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline

# Load the fine-tuned model and tokenizer
new_model = AutoModelForSequenceClassification.from_pretrained('my-fine-tuned-bert')
new_tokenizer = AutoTokenizer.from_pretrained('my-fine-tuned-bert')

# Create a classification pipeline
classifier = TextClassificationPipeline(model=new_model, tokenizer=new_tokenizer)

# Add label mapping for sentiment analysis (assuming LABEL_0 = 'negative' and LABEL_1 = 'positive')
label_mapping = {0: 'negative', 1: 'positive'}

# Test the model
result = classifier("This movie was excellent.")

# Map the result to more meaningful labels
mapped_result = {'label': label_mapping[int(result[0]['label'].split('_')[1])], 'score': result[0]['score']}
print(mapped_result)
